<a href="https://colab.research.google.com/github/AbhishekSinghKushwah7/Plant-Disease-Identification-via-Deep-Learning/blob/main/thermal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Experiment 2**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

In [3]:
def create_directories(output_dir, splits, classes):
    for split in splits:
        for cls in classes:
            os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

def list_all_images(source_dir, classes):
    all_images = []
    for cls in classes:
        class_dir = os.path.join(source_dir, cls)
        images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]
        all_images.extend([(img, cls) for img in images])
    return all_images

def split_dataset(all_images):
    train_val_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)
    train_images, val_images = train_test_split(train_val_images, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
    return train_images, val_images, test_images

def copy_images(images, output_dir, split):
    for img_path, cls in images:
        dest_dir = os.path.join(output_dir, split, cls)
        shutil.copy(img_path, dest_dir)

def prepare_data(source_dir, output_dir, classes):
    splits = ['train', 'validation', 'test']
    create_directories(output_dir, splits, classes)
    all_images = list_all_images(source_dir, classes)
    train_images, val_images, test_images = split_dataset(all_images)
    copy_images(train_images, output_dir, 'train')
    copy_images(val_images, output_dir, 'validation')
    copy_images(test_images, output_dir, 'test')
    return len(train_images), len(val_images), len(test_images)

def build_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

    # Unfreeze more layers for fine-tuning
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    # Add custom layers on top of the base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(x)  # Added L2 regularization and weight init
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-2), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def plot_history(history, test_accuracy=None):
    plt.figure(figsize=(10, 4))  # Reduced size for side-by-side plots

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    if test_accuracy is not None:
        plt.axhline(y=test_accuracy, color='r', linestyle='--', label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training, Validation, and Test Accuracy')

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.tight_layout()
    plt.show()


In [4]:
source_dir = "/content/drive/MyDrive/Colab Notebooks/thermal images UL"
output_dir = "/content/drive/MyDrive/Colab Notebooks/thermal images UL"
classes = ['healthy', 'Blast', 'BLB', 'hispa', 'leaf folder', 'leaf spot']

In [5]:
# Prepare data
train_count, val_count, test_count = prepare_data(source_dir, output_dir, classes)
print(f"Training set: {train_count} images")
print(f"Validation set: {val_count} images")
print(f"Test set: {test_count} images")


Training set: 381 images
Validation set: 127 images
Test set: 128 images


In [6]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,  # Reduced
    width_shift_range=0.1,  # Reduced
    height_shift_range=0.1,  # Reduced
    shear_range=0.1,  # Reduced
    zoom_range=0.1,  # Reduced
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=128,  # Increased batch size
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'validation'),
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'test'),
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

Found 540 images belonging to 8 classes.
Found 227 images belonging to 8 classes.
Found 232 images belonging to 8 classes.


In [7]:
# Build the Model
model = build_resnet_model(input_shape=(224, 224, 3), num_classes=train_generator.num_classes)


In [ ]:
# Training the Model
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5, verbose=1)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


In [ ]:
# Evaluate the Model
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]
print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Accuracy: {val_accuracy}")

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Testing Accuracy: {test_accuracy}")

In [ ]:
# Plot Training/Validation Accuracy and Loss
plot_history(history, test_accuracy=test_accuracy)

In [ ]:
# Save the Model
model.save('resnet_image_classification_model.h5')
print("Model saved as 'resnet_image_classification_model.h5'")